# Task 02 - kodelabs

## import dependencies

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

## Load and preprocess the dataset

In [44]:
dataset = pd.read_csv("dataset.csv")
dataset['Examples'] = dataset['Examples'].str.lower().str.replace('[^\w\s]', '', regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\User\AppData\Local\Temp\ipykernel_980\3253928910.py:2: SyntaxWarning: invalid escape sequence '\w'
  dataset['Examples'] = dataset['Examples'].str.lower().str.replace('[^\w\s]', '', regex=True)


## Tokenize and Labels

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['Examples'])
max_len = max(len(seq) for seq in tokenizer.texts_to_sequences(dataset['Examples']))
X_seq = tokenizer.texts_to_sequences(dataset['Examples'])
X_pad = pad_sequences(X_seq, maxlen=max_len, padding='post')

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(dataset['Intent'])


## Build Train and Test

In [46]:
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_len),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_pad, y_encoded, epochs=100, batch_size=100) 

def classify_intent(text, threshold=0.7):
    text = text.lower().replace('[^\w\s]', '')
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    confidence = np.max(model.predict(padded_sequence))
    if confidence >= threshold:
        prediction = np.argmax(model.predict(padded_sequence))
        intent = label_encoder.inverse_transform([prediction])[0]
        return intent, confidence
    else:
        return "NLU fallback: Intent could not be confidently determined", confidence

test_texts = ["This is the reason for it", "Stay hydrated", "How about we experiment with new ideas?", "This isn't what I ordered"]
for text in test_texts:
    intent, confidence = classify_intent(text)
    print(f"Text: '{text}'")
    print(f"Intent: {intent}, Confidence: {confidence:.4f}")
    print()


<>:16: SyntaxWarning: invalid escape sequence '\w'
<>:16: SyntaxWarning: invalid escape sequence '\w'
C:\Users\User\AppData\Local\Temp\ipykernel_980\965641953.py:16: SyntaxWarning: invalid escape sequence '\w'
  text = text.lower().replace('[^\w\s]', '')
c:\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - accuracy: 0.1056 - loss: 2.7753
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2726 - loss: 2.4945
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4104 - loss: 2.1319
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5367 - loss: 1.6643
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5820 - loss: 1.4023
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6791 - loss: 1.0671
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.7343 - loss: 0.9178
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7914 - loss: 0.7378
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8567 - loss: 0.5997
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8794 - loss: 0.4820
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9290 - loss: 0.3644
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9408 - lo